In [1]:
import numpy as np
import pandas as pd

import os

import matplotlib.pyplot as plt

import cv2 as cv

from sklearn.model_selection import train_test_split 

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications import VGG16

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout

from tensorflow.keras.models import save_model, load_model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

In [2]:
power_10_df = pd.read_csv("C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Power measurements\\2019_09_10.csv")
power_11_df = pd.read_csv("C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Power measurements\\2019_09_11.csv")
power_12_df = pd.read_csv("C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Power measurements\\2019_09_12.csv")

power_10_df['Date and Time'] = pd.to_datetime(power_10_df['Date and Time'])
power_11_df['Date and Time'] = pd.to_datetime(power_11_df['Date and Time'])
power_12_df['Date and Time'] = pd.to_datetime(power_12_df['Date and Time'])

power_df = pd.concat([power_10_df, power_11_df, power_12_df], axis = 0)
power_df.dropna(inplace=True)

power_df_avg = power_df.groupby('Date and Time').mean()
power_df_avg          

power_df_avg['DateTime'] = power_df_avg.index
power_df_avg = power_df_avg.reset_index(drop=True)
power_df_avg       

C:\Users\user\AppData\Local\Temp\ipykernel_22340\1493777463.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  power_10_df['Date and Time'] = pd.to_datetime(power_10_df['Date and Time'])
C:\Users\user\AppData\Local\Temp\ipykernel_22340\1493777463.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  power_11_df['Date and Time'] = pd.to_datetime(power_11_df['Date and Time'])
C:\Users\user\AppData\Local\Temp\ipykernel_22340\1493777463.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  power_12_df['Date and Time'] = pd.to_datetime(power_12_df['Date and Time'])


,AVG power,DateTime
0,0.0,2019-09-10 00:00:00
1,0.0,2019-09-10 00:01:00
2,0.0,2019-09-10 00:02:00
3,0.0,2019-09-10 00:03:00
4,0.0,2019-09-10 00:04:00
...,...,...
4083,0.0,2019-09-12 20:03:00
4084,0.0,2019-09-12 20:04:00
4085,0.0,2019-09-12 20:05:00
4086,0.0,2019-09-12 20:06:00


In [3]:
time_10_df = pd.read_csv("C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\10_09_2019\\Time.csv")
time_11_df = pd.read_csv("C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\11_09_2019\\Time.csv")
time_12_df = pd.read_csv("C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\12_09_2019\\Time.csv")

time_10_df = time_10_df[:-1]
time_11_df = time_11_df[:-1]
time_12_df = time_12_df[:-1]

time_10_df['Date and Time'] = pd.to_datetime(time_10_df[['Hours', 'Minutes', 'Seconds']].assign(Day=10, Month=9, Year=2019))
time_10_df = time_10_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)

time_11_df['Date and Time'] = pd.to_datetime(time_11_df[['Hours', 'Minutes', 'Seconds']].assign(Day=11, Month=9, Year=2019))
time_11_df = time_11_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)

time_12_df['Date and Time'] = pd.to_datetime(time_12_df[['Hours', 'Minutes', 'Seconds']].assign(Day=12, Month=9, Year=2019))
time_12_df = time_12_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)

folder = "C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\"

time_10_df['FilePath'] = [os.path.join(folder, '10_09_2019\\Images\\' + str(image_number) + '.jpg') for image_number in time_10_df['Image Number']]
time_11_df['FilePath'] = [os.path.join(folder, '11_09_2019\\Images\\' + str(image_number) + '.jpg') for image_number in time_10_df['Image Number']]
time_12_df['FilePath'] = [os.path.join(folder, '12_09_2019\\Images\\' + str(image_number) + '.jpg') for image_number in time_10_df['Image Number']]

time_10_df.dropna(inplace=True)
time_11_df.dropna(inplace=True)
time_12_df.dropna(inplace=True)

In [4]:
time_10_df['DateTime'] = time_10_df['Date and Time'].dt.floor('min')
time_11_df['DateTime'] = time_11_df['Date and Time'].dt.floor('min')
time_12_df['DateTime'] = time_12_df['Date and Time'].dt.floor('min')

grouped = time_10_df.groupby('DateTime')
grouped_11 = time_11_df.groupby('DateTime')
grouped_12 = time_12_df.groupby('DateTime')

aggregated = grouped['FilePath'].agg(lambda x: ','.join(x))
aggregated_11 = grouped_11['FilePath'].agg(lambda x: ','.join(x))
aggregated_12 = grouped_12['FilePath'].agg(lambda x: ','.join(x))

result = aggregated.reset_index().rename(columns={'FilePath': 'FilePaths'})
result_11 = aggregated_11.reset_index().rename(columns={'FilePath': 'FilePaths'})
result_12 = aggregated_12.reset_index().rename(columns={'FilePath': 'FilePaths'})

result['NumberFiles'] = result['FilePaths'].str.split(',').str.len()
result_11['NumberFiles'] = result_11['FilePaths'].str.split(',').str.len()
result_12['NumberFiles'] = result_12['FilePaths'].str.split(',').str.len()

images_df = pd.concat([result, result_11, result_12], axis = 0)
images_df

,DateTime,FilePaths,NumberFiles
0,2019-09-10 08:00:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,5
1,2019-09-10 08:01:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
2,2019-09-10 08:02:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
3,2019-09-10 08:03:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
4,2019-09-10 08:04:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
...,...,...,...
496,2019-09-12 16:16:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
497,2019-09-12 16:17:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
498,2019-09-12 16:18:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6
499,2019-09-12 16:19:00,C:\Users\user\Desktop\sprint_test_II\_data_ana...,6


In [5]:
merged_df = pd.merge(power_df_avg, images_df, on='DateTime')
merged_df
merged_df['FilePaths'][0]

'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\10_09_2019\\Images\\1.jpg,C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\10_09_2019\\Images\\2.jpg,C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\10_09_2019\\Images\\3.jpg,C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\10_09_2019\\Images\\4.jpg,C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\Low-res sky images kaggle\\Sky images\\10_09_2019\\Images\\5.jpg'

In [6]:
bins = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000]
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
merged_df['category'] = pd.cut(merged_df['AVG power'], bins=bins, labels=labels)

# count_occurrences = np.count_nonzero(merged_df['category'] == '11')
# print(count_occurrences)

In [7]:
file_list= []
for i in range(merged_df.shape[0]):
    for j in range( merged_df['NumberFiles'][i]):
        file=merged_df['FilePaths'][i].split(',')[j]
        file_list.append(file)
len(file_list)

8997

In [8]:
y=[]
for i in range(merged_df.shape[0]):
    for j in range( merged_df['NumberFiles'][i]):
        y_=merged_df['category'][i]
        y.append(y_)
len(y)  

8997

In [9]:
X=np.zeros((8997,150,150,3))

In [10]:
for idx,file in enumerate(file_list):
    dt=plt.imread(file)
    dt=cv.resize(dt,(150,150)).reshape(1,150,150,-1)
    X[idx] = dt[:,:,:,:3]

In [11]:
X_, X_test, y_, y_test=train_test_split(X,y,test_size=.2)
X_train, X_validation, y_train, y_validation=train_test_split(X_,y_)

In [12]:
y_train = to_categorical(y_train)
y_validation = to_categorical(y_validation)
y_test = to_categorical(y_test)

In [13]:
X_train[0].shape, y_train[0].shape

((150, 150, 3), (13,))

In [14]:
conv_base=VGG16(include_top=False, input_shape=(150,150,3))

In [17]:
# 훈련 데이터에 대한 데이터 증강
train_datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True, vertical_flip=True)
train_data = train_datagen.flow(X_train, y_train, batch_size=128)

# 검증 및 테스트 데이터는 증강을 적용하지 않음
validation_datagen = ImageDataGenerator()
validation_data = validation_datagen.flow(X_validation, y_validation, batch_size=128)

test_datagen = ImageDataGenerator()
test_data = test_datagen.flow(X_test, y_test, batch_size=64)


# ImageDataGen=ImageDataGenerator(rotation_range=20, horizontal_flip=True,vertical_flip=True) 

# train_data=ImageDataGen.flow(X_train, y_train, batch_size=64)
# validation_data=ImageDataGen.flow(X_validation, y_validation, batch_size=32)
# test_data=ImageDataGen.flow(X_test, y_test, batch_size=32)

In [18]:
X_train_trans=conv_base.predict(X_train)
X_validation_trans=conv_base.predict(X_validation)

57/57 [==============================] - 35s 617ms/step


In [19]:
X_train_trans.shape

(5397, 4, 4, 512)

In [20]:
model = Sequential([
    Flatten(input_shape=X_train_trans.shape[1:]),
    Dense(518, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),  
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(), 
    Dense(13, activation='softmax')
])

In [21]:
optimizer = Adam(learning_rate=0.001)
loss='categorical_crossentropy'
metrics='accuracy'
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [22]:
y_validation.shape

(1800, 13)

In [23]:
model.fit(X_train_trans,y_train,batch_size=128,validation_data=(X_validation_trans,y_validation),epochs=200)

Epoch 1/200


43/43 [==============================] - 6s 53ms/step - loss: 2.2806 - accuracy: 0.3011 - val_loss: 2.3855 - val_accuracy: 0.2839
Epoch 2/200
43/43 [==============================] - 2s 47ms/step - loss: 1.6920 - accuracy: 0.4777 - val_loss: 1.6767 - val_accuracy: 0.4772
Epoch 3/200
43/43 [==============================] - 2s 47ms/step - loss: 1.3983 - accuracy: 0.5553 - val_loss: 1.1264 - val_accuracy: 0.6228
Epoch 4/200
43/43 [==============================] - 2s 47ms/step - loss: 1.1813 - accuracy: 0.6165 - val_loss: 0.9470 - val_accuracy: 0.6689
Epoch 5/200
43/43 [==============================] - 2s 46ms/step - loss: 1.0381 - accuracy: 0.6476 - val_loss: 0.8162 - val_accuracy: 0.7094
Epoch 6/200
43/43 [==============================] - 2s 46ms/step - loss: 0.9312 - accuracy: 0.6841 - val_loss: 0.7406 - val_accuracy: 0.7344
Epoch 7/200
43/43 [==============================] - 2s 48ms/step - loss: 0.8440 - accuracy: 0.7163 - val_loss: 0.6761 - val_accuracy: 0.7761
Epoc

In [24]:
model.evaluate(conv_base.predict(X_test),y_test)

57/57 [==============================] - 0s 6ms/step - loss: 0.3046 - accuracy: 0.9289


[0.3045794367790222, 0.9288889169692993]

In [27]:
save_model(model, 'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\bins14_model.h5')

C:\Users\user\AppData\Local\Temp\ipykernel_22340\1142756231.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\bins14_model.h5')


In [28]:
save_model(conv_base,'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\VGG16_14_model.h5')

C:\Users\user\AppData\Local\Temp\ipykernel_22340\1927730054.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(conv_base,'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\VGG16_14_model.h5')


In [29]:
new_model = load_model('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\bins14_model.h5')

In [30]:
new_conv = load_model('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\VGG16_14_model.h5')

In [31]:
labels_dic = {
    "0to500" : 0
    ,"501to1000" : 1
    ,"1001to1500" : 2
    ,"1501to2000" : 3
    ,"2001to2500" : 4
    ,"2501to3000" : 5
    ,"3001to3501" : 6
    ,"4000to4500" : 7
    ,"4501to5000" : 8
    ,"5001to5500" : 9
    ,"5501to6000" : 10
    ,"6001to6500" : 11
    ,"6501to7000" : 12
    ,"7001to7500" : 13
    ,"7501to8000" : 14
}

labels_dic = {v:k for k,v in labels_dic.items()} 

In [32]:
# x=plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\22_01_2024\\9.jpg')
# x = cv.resize(x,(150,150))
# x = x[:,:,:3]
# x = x.reshape(1,150,150,3)

# conv_base = VGG16(include_top=False, input_shape=(150,150,3))
# x_trans=conv_base.predict(x)

# new_model.predict(x_trans)
# np.argmax(new_model.predict(x_trans))


In [33]:
def model_predict(x, conv_base):
    x1 = cv.resize(x,(150,150))
    x1 = x1[:,:,:3]
    x1 = x1.reshape(1,150,150,3)
    answer=np.argmax(new_model.predict(conv_base.predict(x1)))
    return answer

In [50]:
x=plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\22_01_2024\\13.jpg')
model_predict(x, new_conv)

1/1 [==============================] - 0s 17ms/step


12

In [51]:
x10 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\10.jpg')
x11 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\11.jpg')
x12 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\12.jpg')
x13 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\13.jpg')
x14 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\14.jpg')
x15 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\15.jpg')
x16 = plt.imread('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\test_photo\\19_01_2024\\16.jpg')

In [52]:
print(model_predict(x10, new_conv))
print(model_predict(x11, new_conv))
print(model_predict(x12, new_conv))
print(model_predict(x13, new_conv))
print(model_predict(x14, new_conv))
print(model_predict(x15, new_conv))
print(model_predict(x16, new_conv))

1/1 [==============================] - 0s 17ms/step
2
1/1 [==============================] - 0s 20ms/step
2
1/1 [==============================] - 0s 21ms/step
2
1/1 [==============================] - 0s 19ms/step
2
1/1 [==============================] - 0s 19ms/step
2
1/1 [==============================] - 0s 20ms/step
2
1/1 [==============================] - 0s 20ms/step
2


In [53]:
x11 = plt.imread("C:/Users/user/Downloads/sky-988796_1280.jpg")
print(model_predict(x11, new_conv))

1/1 [==============================] - 0s 17ms/step
10
